In [5]:
import pandas as pd
import numpy as np
import collections
import geopandas
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

In [193]:
plt.rcParams["figure.figsize"] = (16,16)

In [27]:
# DO NOT RUN!!!!
# DO NOT RUN!!!!
df = pd.read_excel (r'FULL_PAS_DATA_ROMAN_CLEAN.xlsm', sheet_name = 'FULL_PAS_DATA_ROMAN')
# DO NOT RUN!!!!
# DO NOT RUN!!!!
# To Do: Add the hoard/coin hoards rows

In [196]:
# The dataset but without completely empty columns
D = pd.DataFrame(df, columns= ["id",
"old_findID",
"uniqueID",
"objecttype",
"classification",
"subclass",
"length",
"height",
"width",
"weight",
"thickness",
"diameter",
"quantity",
"otherRef",
"curr_loc",
"discoveryMethod",
"treasureID",
"broadperiod",
"numdate1",
"numdate2",
"description",
"notes",
"reuse",
"reusePeriodID",
"created",
"updated",
"secwfstage",
"findofnote",
"objecttypecert",
"datefound1",
"datefound2",
"inscription",
"museumAccession",
"subsequentAction",
"objectCertainty",
"dateFromCertainty",
"dateToCertainty",
"dateFoundFromCertainty",
"dateFoundToCertainty",
"subPeriodFrom",
"subPeriodTo",
"objdate1period",
"objdate2period",
"secuid",
"material1",
"material2",
"manmethod",
"decmethod",
"decstyle",
"complete",
"surface",
"manufactureID",
"culture",
"recorderID",
"identifier1ID",
"identifier2ID",
"smrRef",
"createdBy",
"updatedBy",
"hoardcontainer",
"institution",
"reason",
"fullname",
"primaryMaterial",
"primaryBMmaterial",
"secondaryMaterial",
"secondaryBMmaterial",
"decoration",
"style",
"manufacture",
"surfaceTreatment",
"completeness",
"preservation",
"periodFrom",
"seneschalPeriodFrom",
"bmPeriodFrom",
"periodoPeriodFrom",
"periodTo",
"seneschalPeriodTo",
"bmPeriodTo",
"periodoPeriodTo",
"reusePeriod",
"ascribedCulture",
#"culturePeriodo", only possible values are [nan 'p08m57hqxg6' 'p08m57hcbrw']
#"cultureBM", only possible values are [nan 'x41012' 'x41145']
"discmethod",
"identifier",
"secondaryIdentifier",
"recorder",
"fromCirca",
"toCirca",
"findSpotID",
"countyID",
"districtID",
"regionID",
"knownas",
"gridlen",
"accuracy",
"source",
"coinID",
"obverseDescription",
"obverseInscription",
"reverseDescription",
"reverseInscription",
#"cciNumber", 100% Null
"denominationID",
"degreeOfWear",
"allenType",
#"vaType", 100% Null
#"mackType", 100% Null
#"abcType", 100% Null
#"bmcType", 100% Null
"reeceID",
"dieAxis",
"moneyer",
"revtypeID",
"categoryID",
"typeID",
"tribeID",
"statusID",
"rulerQualifier",
"denominationQualifier",
"mintQualifier",
"dieAxisCertainty",
"initialMark",
"reverseMintMark",
"statusQualifier",
"ruler1",
#"ruler2", only possible values are [nan  0.]
"mintID",
"rrcID",
"ricID",
"tribe",
"ironAgeRegion",
"ironAgeArea",
"denomination",
"nomismaDenomination",
#"dbpediaDenomination", 100% Null
#"bmDenomination", 100% Null
"primaryRuler",
"viaf",
"rulerDbpedia",
"nomismaRulerID",
#"secondaryRuler", 100% Null
"periodName",
"dateRange",
"mintName",
"nomismaMintID",
"pleiadesID",
"mintGeonamesID",
"mintWoeid",
"mintOsID",
"mintGettyID",
"mintWoeID",
"mintDbPediaID",
"mintWhat3Words",
"mintBritMuseumID",
"wear",
"nomismaWear",
"dieAxisName",
"category",
"type",
"moneyerID",
"nomismaMoneyer",
"emperorID",
"romanMintID",
"reverseType",
"status",
#"jettonClass", 100% Null
#"jettonType", 100% Null
#"jettonGroup", 100% Null
"thumbnail",
"filename",
"filesize",
"imageLabel",
"imageCopyrightHolder",
#"imageLicense", only possible values are [nan 'Attribution-ShareAlike License' 'Attribution License']
"imagedir",
"region",
"rallyID",
"rallyName",
"rallyDateFrom",
"rallyDateTo",
"landuse",
"landvalue",
#"regionType", only possible values are ['European Region' nan]
"countyType",
"county",
"districtType",
"district",
"parishType",
"subsequentActionTerm",
#"bmThesObject", only possible values are [6089.   nan 5104.]
"seneschalObject",
"parishID",
"fourFigure",
"map25k",
"map10k",
"fourFigureLat",
"fourFigureLon",
"woeid",
#"geonamesID", only possible values are [nan 'Not Found']
"what3words",
"centreLat",
"centreLon",
"parish",
"URL Page"])
# Get rid of all my custom-inserted values and use the pandas "NaN" instead
D = D.replace("Not Found", np.nan)

In [122]:
# Determine which columns are likely useless for the thesis analysis
#x = D.isnull().sum()

In [123]:
# Was originally used to remove completely empty columns
#for item in D:
#    if x[item] == 334855:
#        print(item + "   " + str(x[item]))


In [124]:
# Was originally used to check for columns with trivial information
#for column in D:
#    if D[column].nunique() <= 5:
#        print(str("Column: " + column + "   " + str(D[column].nunique())))
#        print(D[column].unique())
#        print("")

In [125]:
# Select only brooch-related objects
BROOCHES = ["BROOCH", "BOW BROOCH", "BROOCH PIN", "PLATE BROOCH", "PENANNULAR BROOCH"]
Brooch_DF = D.loc[lambda D: D['objecttype'].isin(BROOCHES)]

In [126]:
#CNT = collections.Counter()
#for item in Brooch_DF["classification"]:
#    CNT[item] += 1
#print(CNT)

In [402]:
#------------------------------------------- MAPPING --------------------------------------------------------
# Download the basemap shapefile
UK_Map = geopandas.read_file("UK Shapefiles/gadm36_GBR_2.shp")

# assumes current_year is >= start date
def calc_transparency(start, end, current_year):
    if current_year >= end:
        return 1.0
    else:
        #(e.g. for range 43-53) (at 43 we want 1/10, and at 53 we want 10/10 --> anything beyond should be 10/10)
        total_range = float(end - start) #denomenator 
        numerator = float(current_year - start + 1)
        return float(numerator/total_range)

# Define the function for plotting a set of points, using "Target_DF"
# Creates a transparency-adjusted map
def create_map(Target_DF, saveas, year_displayed):
    gdf = geopandas.GeoDataFrame(
        Target_DF, geometry=geopandas.points_from_xy(Target_DF.fourFigureLon, Target_DF.fourFigureLat))
    list_of_transparencies = []
    for index, row in Target_DF.iterrows():
        list_of_transparencies.append(calc_transparency(row["numdate1"], row["numdate2"], year_displayed))
        
    dotcolors=[(1.0, 0.0, 0.0, a) for a in list_of_transparencies]
    
    gdf.plot(ax = UK_Map.plot(color='white', edgecolor = 'black'), 
             c=dotcolors, markersize=1)
    
    plt.text(0, 56, (str(year_displayed) + " AD"), size = 25, color='red')
    plt.savefig(saveas, bbox_inches="tight")

In [405]:
# Objecttype to map
main_dataframe = Brooch_DF
stem = "Brooches_"

def animate_map(main_DF):
    # For years greater than 0...
    MD = main_DF[main_DF.numdate1 > 0]
    frame = 1
    # range of years to animate
    #for i in [1, 25, 40, 43, 50, 55, 60, 70, 75, 80, 100, 150, 200, 250, 300, 320, 400]:
    for i in range(1,400):
        # Create mini dataframe
        Temporary_DF = MD[MD.numdate1 <= i]
        fig_title = (stem + "frame-%s" % ('{:0>5}'.format(frame),))
        create_map(Temporary_DF, fig_title, i)
        frame+=1
        
    
# Run this command in the terminal to turn into video
#                       ^ Frames per second
# ffmpeg -f image2 -r 1/1 -i Brooches_frame-%05d.png -vcodec mpeg4 -y movie1.mp4
# ffmpeg -pattern_type glob -i "Brooches_year-*.png" video.webm


# Once the above is working, change transparency of dots according to date range

In [426]:
#animate_map(main_dataframe)

In [271]:
#CNT = collections.Counter()
#for index, row in Brooch_DF.iterrows():
#    range_start = row['numdate1']
#    range_end = row['numdate2']
#    range_size = range_end - range_start
#    if not np.isnan(range_size):
#        CNT[range_size] +=1
#        if range_size > 700:
#            print("Start: " + str(range_start) + " End: " + str(range_end))
#print(CNT)
    #print(row['numdate1'], row['numdate2'])

In [328]:
for i in range(1, 600):
    temp = Brooch_DF[Brooch_DF.numdate1 == i]
#temp2 = temp[temp.numdate1 == 1]
    print(str(i) + ", " + str(temp.id.count()))

1, 1219
2, 8
3, 0
4, 1
5, 0
6, 0
7, 0
8, 0
9, 0
10, 22
11, 0
12, 0
13, 0
14, 1
15, 10
16, 0
17, 0
18, 0
19, 0
20, 156
21, 0
22, 0
23, 0
24, 0
25, 569
26, 0
27, 0
28, 0
29, 0
30, 50
31, 0
32, 2
33, 0
34, 0
35, 0
36, 0
37, 0
38, 0
39, 0
40, 625
41, 8
42, 5
43, 8960
44, 10
45, 140
46, 1
47, 2
48, 0
49, 4
50, 2847
51, 1
52, 0
53, 0
54, 2
55, 532
56, 0
57, 0
58, 0
59, 0
60, 302
61, 0
62, 2
63, 2
64, 0
65, 61
66, 0
67, 0
68, 0
69, 83
70, 793
71, 0
72, 0
73, 0
74, 0
75, 4213
76, 0
77, 1
78, 1
79, 0
80, 1897
81, 0
82, 1
83, 0
84, 0
85, 18
86, 0
87, 0
88, 0
89, 0
90, 23
91, 0
92, 0
93, 0
94, 0
95, 1
96, 0
97, 0
98, 0
99, 4
100, 4349
101, 58
102, 0
103, 0
104, 0
105, 0
106, 0
107, 0
108, 0
109, 0
110, 7
111, 0
112, 0
113, 0
114, 0
115, 1
116, 0
117, 0
118, 0
119, 1
120, 28
121, 0
122, 0
123, 0
124, 0
125, 11
126, 0
127, 0
128, 0
129, 0
130, 22
131, 0
132, 0
133, 0
134, 0
135, 0
136, 0
137, 0
138, 3
139, 0
140, 13
141, 0
142, 0
143, 0
144, 0
145, 0
146, 0
147, 0
148, 0
149, 0
150, 563
151, 0
152,

In [425]:
CNT = collections.Counter()
for item in D.what3words:
    if not pd.isnull(item):
        for x in item.split('.'):
            CNT[x] += 1
print(CNT)

Counter({'ultra': 661, 'increment': 636, 'successor': 627, 'tops': 379, 'cabbages': 378, 'geek': 373, 'beams': 373, 'bead': 368, 'known': 368, 'sugars': 367, 'drape': 365, 'joystick': 338, 'dinner': 311, 'ever': 303, 'hurls': 302, 'warns': 301, 'immune': 301, 'harsh': 297, 'worm': 297, 'charmingly': 292, 'heartless': 291, 'obstruct': 291, 'cutaway': 291, 'trespass': 284, 'inventors': 282, 'army': 279, 'examples': 278, 'squeaking': 275, 'timer': 275, 'column': 274, 'cycle': 257, 'deployed': 255, 'fail': 255, 'quoted': 255, 'someone': 248, 'helped': 244, 'support': 241, 'simulates': 240, 'mindset': 239, 'planting': 237, 'rugs': 234, 'notch': 233, 'grocers': 230, 'unwell': 227, 'wiggling': 225, 'zinc': 209, 'important': 202, 'safely': 201, 'mural': 200, 'asks': 198, 'region': 198, 'workouts': 197, 'secondly': 196, 'soccer': 194, 'barstool': 192, 'goat': 191, 'skewing': 189, 'plan': 189, 'breeze': 189, 'playfully': 188, 'professes': 187, 'plunge': 185, 'shuttered': 184, 'communal': 183, 'g